In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

sys.path.append('../')
from models.model import cnn

In [3]:
arch_str = """
c1(och 2, ker 3, str 1, pad 1, bias, pt avg, pk 2, ps 1); 
fc(out 5, bias, actv tanh);
fc(out 1, bias)
"""

### make test input

In [4]:
test_in = torch.ones((2,2,5)).float()
test_in[1,:,:] = 2.0
print(test_in)

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.]]])


In [23]:
test_in = torch.rand((2,2,5)).float()
# test_in[1,:,:] = 2.0
print(test_in)

tensor([[[0.7773, 0.4208, 0.1714, 0.7680, 0.9926],
         [0.8560, 0.7465, 0.1259, 0.6079, 0.7602]],

        [[0.8552, 0.4199, 0.1301, 0.9309, 0.0832],
         [0.5147, 0.5509, 0.9046, 0.5909, 1.0000]]])


### (optional) make test output

In [5]:
test_out = torch.ones((2,1))*np.array([[-10],[10]])
test_out = test_out.float()
print(test_out)

tensor([[-10.],
        [ 10.]])


### Make network

In [24]:
test_net = cnn(arch_str, 2, 5)

input to conv1D of 2 channels * 5 signal length (10 values)
output of conv1D of 2 channels * 4 signal length (8 values)
output of linear layer of 5 values
output of linear layer of 1 values
--Return--
> /home/scratch/youngsec/rs/fusion/dynamics_models/NNKit/models/model.py(441)__init__()->None
-> import pdb; pdb.set_trace()
(Pdb) c


In [25]:
test_net

cnn(
  (loss): MSELoss()
  (layers): ModuleList(
    (0): Conv1DLayer(
      (pool): AvgPool1d(kernel_size=(2,), stride=(1,), padding=(0,))
    )
    (1): LinearLayer(
      (activation): Tanh()
    )
    (2): LinearLayer()
  )
)

### Manually set weights of network

In [8]:
# set conv layer weights
w_shape = list(test_net.layers[0].weight.shape)
w_size = np.product(w_shape)
print(w_size, w_shape)
b_shape = list(test_net.layers[0].bias.shape)
b_size = np.product(b_shape)
print(b_size, b_shape)

test_net.layers[0].weight = nn.Parameter(torch.arange(w_size).view(w_shape))
test_net.layers[0].bias = nn.Parameter(torch.arange(1,b_size+1).view(b_shape)/10.)

print(test_net.layers[0].weight)
print(test_net.layers[0].bias)

12 [2, 2, 3]
2 [2]
Parameter containing:
tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.]],

        [[ 6.,  7.,  8.],
         [ 9., 10., 11.]]], requires_grad=True)
Parameter containing:
tensor([0.1000, 0.2000], requires_grad=True)


In [9]:
# set conv layer weights
w_shape = list(test_net.layers[1].weight.shape)
w_size = np.product(w_shape)
print(w_size, w_shape)
b_shape = list(test_net.layers[1].bias.shape)
b_size = np.product(b_shape)
print(b_size, b_shape)

test_net.layers[1].weight = nn.Parameter(torch.arange(w_size).view(w_shape))
test_net.layers[1].bias = nn.Parameter(torch.arange(1,b_size+1).view(b_shape)/10.)

print(test_net.layers[1].weight)
print(test_net.layers[1].bias)

40 [5, 8]
5 [5]
Parameter containing:
tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.],
        [16., 17., 18., 19., 20., 21., 22., 23.],
        [24., 25., 26., 27., 28., 29., 30., 31.],
        [32., 33., 34., 35., 36., 37., 38., 39.]], requires_grad=True)
Parameter containing:
tensor([0.1000, 0.2000, 0.3000, 0.4000, 0.5000], requires_grad=True)


### Print layers of network

In [26]:
for l in test_net.layers:
    print('LAYER')
    for x in l.parameters(): print(x)
    print('\n\n')

LAYER
Parameter containing:
tensor([[[ 0.0244, -0.2529, -0.2012],
         [-0.2885, -0.0662, -0.1659]],

        [[ 0.1932,  0.3786,  0.1079],
         [-0.1345,  0.2772, -0.2146]]], requires_grad=True)
Parameter containing:
tensor([0.1756, 0.3224], requires_grad=True)



LAYER
Parameter containing:
tensor([[ 0.0568, -0.1025,  0.0888,  0.2366,  0.0478, -0.3074, -0.2672, -0.2915],
        [ 0.2880, -0.1745,  0.0342, -0.0862,  0.2396,  0.0882, -0.0861,  0.1680],
        [ 0.3023, -0.1783, -0.2027, -0.0726, -0.3350, -0.2648,  0.2378, -0.1394],
        [-0.0159,  0.0503, -0.2911, -0.1049, -0.0697,  0.3455,  0.0080,  0.0325],
        [ 0.2014, -0.2582,  0.1819,  0.2187, -0.2137,  0.2059,  0.1928,  0.0958]],
       requires_grad=True)
Parameter containing:
tensor([-0.0315,  0.2167, -0.2266, -0.0667, -0.2653], requires_grad=True)



LAYER
Parameter containing:
tensor([[-0.4282, -0.2514, -0.1093, -0.2895,  0.4367]], requires_grad=True)
Parameter containing:
tensor([-0.2467], requires_grad=Tru

### Print output of network

In [27]:
test_net(test_in)

tensor([[-0.2394],
        [-0.2500]], grad_fn=<AddmmBackward>)

In [12]:
np.mean([1110.1, 2215.1])

1662.6

In [13]:
np.sqrt(np.var([1110.1, 2215.1])+1e-5)

552.5000000090498

In [14]:
np.sqrt(np.var([13.6, 27.1])+1e-5)

6.750000740740702

In [15]:
(13.6-20.35)/6.7

-1.0074626865671643

### (optional) train network to overfit test_out

In [28]:
optimizer = torch.optim.SGD(test_net.parameters(), lr=0.01)

In [29]:
mse = torch.nn.MSELoss()

In [30]:
for ep in range(100):
    preds = test_net(test_in)
    print(preds.shape, test_out.shape)
    loss = mse(preds, test_out)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(preds)

torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2394],
        [-0.2500]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2358],
        [-0.2385]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2323],
        [-0.2275]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2289],
        [-0.2167]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2255],
        [-0.2060]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2223],
        [-0.1953]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2191],
        [-0.1844]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2160],
        [-0.1734]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2131],
        [-0.1619]], grad_fn=<AddmmBackward>)
torch.Size([2, 1]) torch.Size([2, 1])
tensor([[-0.2102],
        [-0.1500]], grad_

In [32]:
optimizer.param_groups[0]['lr']

0.01